#Linux i računarski klasteri - 3. nedelja

Termin pokriva sledeće oblasti:

*   Pipes - koncept i primeri
*   Utilities: cat, grep, sed,...
*   Shell scripting


# 1. deo: Pipes - koncept i primeri

## Uvod

U UNIX-like operativnim sistemima, pipeline (cevovod - po analogiji sa sistemom koji svakodnevno koristimo) predstavlja alat za međuprocesnu komunikaciju korišćenjem prenosa poruka. Pipeline je skup procesa povezanih njihovim standardnim tokovima, tako da se izlazni tekst svakog procesa (```stdout```) direktno prenosi kao ulaz (```stdin```) sledećem. Drugi proces se pokreće dok se prvi proces još uvek izvršava, i izvršavaju se istovremeno.

Pipes u pipelines su tzv. anonimni pipelines (za razliku od named pipelines), gde se podaci koje je napisao jedan proces baferuju od strane operativnog sistema dok ih ne pročita sledeći proces, a ovaj jednosmerni kanal nestaje kada se procesi završe. Standardna shell sintaksa za anonimne cevi je da se navede više komandi, odvojenih vertikalnim crtama (```|``` - pipe u UNIX terminologiji).

![Pipeline primer](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Pipeline.svg/1920px-Pipeline.svg.png)

Tipičan primer korišćenja pipeline‑a je:

```
ls -l | grep key | less
```

U računarstvu, anonimni kanal je simpleks FIFO (simplex - komunikacija u jednom smeru, FIFO - First In First Out) komunikacioni kanal koji se može koristiti za jednosmernu međuprocesnu komunikaciju (Inter Process Communication - IPC). Implementacija je često integrisana u podsistem za unos datoteka operativnog sistema. Tipično, roditeljski program otvara anonimne kanale i kreira novi proces koji nasleđuje druge krajeve kanala ili kreira nekoliko novih procesa i raspoređuje ih u kanal. Dvosmerna (full-duplex) komunikacija obično zahteva dva anonimna kanala.

Sa druge strane, named pipe (takođe poznata kao FIFO zbog svog ponašanja) je proširenje tradicionalnog koncepta pipe u Unix (Unix-like) sistemima i jedna je od metoda međuprocesne komunikacije (IPC). Tradicionalni pipe je „neimenovan“ i traje samo dok traje proces. Međutim, imenovani pipe može trajati dok god je sistem pokrenut, nakon životnog veka procesa. Može se izbrisati ako se ne koristi dugo. Obično se imenovana cev pojavljuje kao datoteka i generalno se procesi priključuju na nju radi IPC-a.




## Da li je pipe program?

Posmatrajući iz perspektive bash-a, pipe (```|``` zajedno sa ```|&```) ne predstavlja komandu, nego kontrolni operater. Pipeline je sekvenca jedne ili više komandi koje su razvojene jednim od kontrolnih operatera.

Primeri:
- Input

```
|

```
- Output:

```
-bash: syntax error near unexpected token `|'

```
- Input:

```
a|b
```
pri čemu su a i b nepoznate komande.

- Output:

```
-bash: b: command not found
-bash: a: command not found
```
bez spominjanja with `|` iz razloga što pipe funkcioniše normalno kada se koristi između bilo koje dve proizvoljne komande. Trenutni problem je što te dve komande ne postoje. Pipe operator govori shell-u da komande koje spaja treba da budu organizovane u pipeline. Shell pokreće obe komande po standardnoj proceduri da je input naredne output prošle komande.

## Rezervisane reči i termini

Rezervisane reči predstavljaju termine koji imaju specijalno značenje u shell-u. Oni se koriste kao početak ili kraj složenih shell komandi.

Sledeće reči su prepoznate kao rezervne kada su iznete bez navodnika i kao prva reč komande:
```
if	then	elif	else	fi	time
for	in	until	while	do	done
case	esac	coproc	select	function
{	}	[[	]]	!
```
```in``` je prepoznat kao rezervisana reč ukoliko je treća reč ```case``` ili ```select``` comande. ```in``` i ```do``` su prepoznate kao rezervne reči ukoliko su treće reči unutar ```for``` komande.


## Složene komande

Složene komande su konstrukcije programskog jezika shell. Svaka konstrukcija počinje rezervisanom rečju ili kontrolnim operatorom i završava se odgovarajućom rezervisanom rečju ili operatorom. Sva preusmeravanja povezana sa složenom komandom primenjuju se na sve komande unutar te složene komande, osim ako nisu eksplicitno zamenjena.

U većini slučajeva, lista komandi u opisu složene komande može biti odvojena od ostatka komande jednim ili više novih redova, a može biti praćena novim redom umesto ```;```.

Bash pruža konstrukcije petlji, uslovne komande i mehanizme za grupisanje komandi i njihovo izvršavanje kao jedinice.

### Primeri

Koristićemo dataset "particle_events_dataset.csv"

1. Petlja kroz sve čestice i broji pojavljivanja
```
for p in proton neutron electron muon tau; do
  echo -n "$p: "
  grep -c "$p" particle_events_dataset.csv
done
```




## Generalni termin - pipeline iz perspektive software development-a

Pipeline predstavlja lanac procesnih elemenata (procesi, thread-ovi, ko-rutine, funkcije,itd.) uređenih na takav način da izlaz (output) svakog elementa predstavlja input za sledeći. Primena ovog generalnog koncepta je predstavljena u prethodnim paragrafima iz perspektive UNIX operativnih sistema. Obično se između uzastopnih elemenata obezbeđuje određena količina baferovanja. Informacije koje teku u ovim pipelines često su tok bajtova ili bitova, a elementi pipeline-a mogu se nazvati filterima. Ovo se takođe naziva obrazac dizajna pipeline-a i filtera koji je monolitan. Njegove prednosti su jednostavnost i niska cena, dok su mu mane nedostatak elastičnosti, tolerancija na greške i skalabilnost. Povezivanje elemenata u pipeline je analogno kompoziciji funkcija. Usko govoreći, pipeline je linearan i jednosmeran, mada se ponekad termin primenjuje na opštije tokove.

## Primeri

1.  Broj redova u fajlu
```
cat podaci.txt | wc -l
```
cat podaci.txt – prikazuje sadržaj fajla podaci.txt.

  * ```wc -l``` – broji broj linija (redova).
  * Korisno za: proveru veličine dataset-a (broj uzoraka).



2. Jedinstvene vrednosti iz kolone CSV fajla
```
cut -d ',' -f2 podaci.csv | sort | uniq
```
  * ```cut -d ',' -f2``` – uzima drugu kolonu CSV fajla (delimiter je zarez).
  * sort – sortira linije (neophodno pre uniq).
  * uniq – prikazuje samo unikatne vrednosti.
  * Korisno za: analizu kategorijskih promenljivih.

3. Brojanje grešaka u log fajlu
```
cat log.txt | grep 'ERROR' | wc -l
```

  * ```grep 'ERROR'``` – filtrira linije koje sadrže "ERROR".

  * ```wc -l``` – broji koliko takvih linija ima.
  * Korisno za: detekciju problema u sistemima i kodovima.

4. Top 10 najčešćih reči u tekstu
```
cat tekst.txt | tr -s ' ' '\n' | sort | uniq -c | sort -nr | head
```

  * ```tr -s ' ' '\n'``` – zamenjuje razmake novim redovima (svaka reč posebno).
  * ```sort``` – sortira reči abecedno.

  * ```uniq -c``` – broji koliko puta se svaka reč pojavljuje.

  * ```sort -nr``` – sortira brojčano, opadajuće.

  * ```head –``` prikazuje prvih 10 rezultata.
  * Korisno za: tekstualnu analizu, frekvencijsku statistiku.

5. Filtriranje vrednosti većih od 1000 u 3. koloni
```
awk -F',' '$3 > 1000' podaci.csv
```

  * ```-F','``` – postavlja zarez kao separator kolona.

  * ```$3 > 1000``` – uslov: treća kolona veća od 1000.
  * Korisno za: selekciju redova u kvantitativnoj analizi.

6. Konverzija CSV → TSV
```
cat podaci.csv | tr ',' '\t'
```
  * ```tr ',' '\t'``` – menja sve zareze u tabove.
  * Korisno za: prebacivanje u format pogodan za alate poput Excel-a, Gnuplot-a.

7. Broj Python fajlova u direktorijumu
```
ls *.py | wc -l
```
  * ```ls *.py``` – prikazuje sve fajlove sa ekstenzijom .py.
  * ```wc -l``` – broji koliko ih ima.
  * Korisno za: praćenje obima projekta.

8. ```TODO``` komentari u kodu
```
grep -r 'TODO' . | less
```
  * ```grep -r``` – rekurzivno traži 'TODO' u svim fajlovima.
  * ```less``` – prikazuje rezultate stranica po stranica.
  * Korisno za: identifikaciju preostalih zadataka u kodu.

9. Top 10 procesa po korišćenju memorije
```
ps aux | sort -nk +4 | tail -n 10
```
  * ```ps aux``` – prikazuje sve aktivne procese.
  * ```sort -nk +4``` – sortira po 4. koloni (memorija), numerički.
  * ```tail -n 10``` – prikazuje 10 najviših vrednosti.
  * Korisno za: monitoring resursa tokom velikih analiza (npr. treniranje modela).

10. Otkrivanje skokova u vremenskim serijama
```
cat data.csv | awk -F',' '{ if ($2 - last > 100) print $0; last=$2 }'
```
  * ```-F',' ``` – postavlja zarez kao separator
  * ```if ($2 - last > 100)``` – proverava da li postoji nagla promena.
  * ```last=$2``` – pamti prethodnu vrednost za sledeću iteraciju.
  * Korisno za: detekciju anomalija u merenjima ili eksperimentima (npr. u CERN-u).

11. Srednja vrednost kolone (npr. za normalizaciju)
```
awk -F',' '{ sum += $2; count++ } END { print sum / count }' podaci.csv
```
  * ```-F','``` – postavlja zarez kao separator (csv format).
  * ```sum += $2``` – sabira sve vrednosti iz druge kolone.
  * ```count++``` – broji koliko ih ima.
  * ```END { print sum / count }``` – nakon čitanja svih redova, ispisuje prosečnu vrednost.
  * Korisno za: standardizaciju podataka pre modeliranja.

12. Raspodela podataka sa Gnuplot-om
```
cut -d',' -f2 podaci.csv | sort -n | gnuplot -p -e 'plot "-" using 1 with lines'
cut -d',' -f2 – izdvaja drugu kolonu.
```
  * ```sort -n``` – sortira numerički.
  * ```gnuplot``` – alat za vizualizaciju; -p ostavlja graf otvoren; -e izvršava inline komandu za crtanje linije.
  * Korisno za: brzi vizuelni uvid u oblik distribucije vrednosti.

13. Detekcija praznih vrednosti u CSV fajlu
```
awk -F',' '{ for (i = 1; i <= NF; i++) if ($i == "") print NR, $0 }' podaci.csv
```
  * ```NF``` – broj polja u redu.

  * ```for``` petlja – prolazi kroz sve kolone.

  * ```if ($i == "")``` – proverava da li je neka kolona prazna.

  * ```NR``` – broj linije (redni broj).
  * Korisno za: validaciju dataset-a pre obrade ili treniranja modela.

14. Pretraga binarnih fajlova većih od 100 MB
```
find . -type f -size +100M | file - | grep 'binary'
find . -type f -size +100M – traži fajlove veće od 100MB.
```
  * ```file``` - – detektuje tip fajla.
  * ```grep 'binary'``` – prikazuje samo binarne fajlove.
  * Korisno za: detekciju velikih fajlova sa eksperimentalnim rezultatima (npr. ROOT fajlovi u CERN-u).

15. Spajanje više CSV fajlova bez duplikata zaglavlja
```
head -n 1 file1.csv > merged.csv && tail -n +2 -q file*.csv >> merged.csv
head -n 1 – uzima samo zaglavlje iz prvog fajla.
```
  * ```tail -n +2``` – preskače zaglavlja u ostalim fajlovima (počevši od linije 2).

  * ```-q ```– tihi režim (ne prikazuje imena fajlova).
  * Korisno za: objedinjavanje eksperimentalnih podataka iz više sesija.

16. Top greške u CERN log fajlu
```
grep 'ERROR' cern.log | cut -d':' -f2 | sort | uniq -c | sort -nr | head
```
  * ```cut -d':' -f2```– uzima opis greške nakon prefiksa.
  * ```uniq -c``` – broji koliko puta se svaka greška pojavljuje.
  * ```sort -nr``` – sortira po učestalosti.
  * Korisno za: sistemsku dijagnostiku softvera za detektore ili akceleratore.

17. Praćenje rasta fajla tokom eksperimenta
```
watch -n 10 'du -h output.dat'
watch -n 10 – izvršava komandu na svakih 10 sekundi.
```
  * ```du -h``` – prikazuje veličinu fajla u human-readable formatu.
  * Korisno za: nadgledanje live stream-ova podataka sa senzora.

18. Detekcija duplikata u eksperimentalnim podacima
```
sort eksperiment.csv | uniq -d
```
  * ```sort``` – sortira fajl (neophodno pre uniq).
  * ```uniq -d``` – prikazuje samo redove koji su duplikati.
  * Korisno za: osiguravanje kvaliteta i jedinstvenosti eksperimentalnih merenja.

19. Prikaz najskorije izmenjenih fajlova
```
find . -type f -printf '%T@ %p\n' | sort -n | tail
```
 * ```-printf '%T@ %p\n'``` – ispisuje vreme izmene (Unix timestamp) i putanju.

 * ```sort -n``` – sortira po vremenu.

 * ```tail``` – prikazuje poslednjih nekoliko najnovijih fajlova.
 * Korisno za: otkrivanje koji fajlovi se menjaju tokom eksperimenta.

20. Analiza latencije između događaja
```
awk -F',' '{ if (last != "") print $1 - last; last=$1 }' timestamps.csv | sort -n | uniq -c
```
 * ```$1 - last```- – računa razliku između vremenskih oznaka (latenciju).
 * ```uniq -c``` – prikazuje učestalost latencija.
 * Korisno za: analizu brzine događaja (npr. čestice u kolajderu).

